# [Create Agent with Azure AI Search](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-azure-ai-search)
Azure AI Search is an enterprise search system for high-performance applications. It integrates with Azure OpenAI Service and Azure Machine Learning, offering advanced search technologies like vector search and full-text search. Ideal for knowledge base insights, information discovery, and automation.<br/><br/>
**IMPORTANT**: in order to create the index leveraging the "Import and Vectorize" wizard, the `Storage Blob Data Contributor` role on the Storage Account is needed for the Azure AI Managed Identity.

## Create a connection to Azure AI Search using CLI

1. First of all, let's check which connections we have, associated to our project mmai-hub04-prj01-fvye:
   ```az ml connection list --resource-group mmai04-rg --workspace-name mmai-hub04-prj01-fvye```
3. Now, create a new yaml file with the configuration for Azure AI Search, using key-based or key-less (as in this case) authentication. **Please note that the "metadata" section must be filled as shown, including the ResourceId that must contain the connection name reporten on line 1**:
```
name: mmai-hub04-fvye-connection-AISearch
type: azure_ai_search
endpoint: https://mmai-hub04-ai-search-fvye.search.windows.net/
is_shared: true
metadata:
  ApiType: Azure
  ResourceId: /subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.Search/searchServices/mmai-hub04-fvye-connection-AISearch
  type: azure_ai_search
  ```

3. Run the command `az ml connection create --file aisearchconnection.yml --resource-group mmai04-rg --workspace-name mmai-hub04-prj01-fvye`

# Constants

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-2024-08-06" # https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview#setup
index_name =  "ms-surface-specs"
project_connection_string = os.environ["PROJECT_CONNECTION_STRING"]

print(f'Project Connection String: <...{project_connection_string[-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


# Create AI Foundry Project Client

In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool # <<<<<<<<<<<<<<< SPECIFIC FOR AZURE AI SEARCH
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

# List existing connections and highlight the "AI Search" ones

In [3]:
conn_list = project_client.connections.list()
conn_id = ""
for conn in conn_list:    
    if conn.connection_type == "CognitiveSearch":
        print(f">>>>>>> AI SEARCH FOUND: {conn}")
        conn_id = conn.id
    else:
        print(conn)

{
 "name": "mmai-hub04-fvye-connection-AIServices_aoai",
 "id": "/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AIServices_aoai",
 "authentication_type": "AAD",
 "connection_type": "ConnectionType.AZURE_OPEN_AI",
 "endpoint_url": "https://mmai-hub04-ai-servicesfvye.openai.azure.com",
 "key": null
 "token_credential": null
}

{
 "name": "mmai-hub04-fvye-connection-AIServices",
 "id": "/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AIServices",
 "authentication_type": "AAD",
 "connection_type": "ConnectionType.AZURE_AI_SERVICES",
 "endpoint_url": "https://mmai-hub04-ai-servicesfvye.cognitiveservices.azure.com",
 "key": null
 "token_credential": null
}

{
 "name": "grounding_with_bing_search_connection

# Chosen AI Search Connection

In [4]:
conn_id

'/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AISearch'

# Initialize `AzureAISearchTool`, adding the connection id to it

In [5]:
ai_search = AzureAISearchTool(index_connection_id=conn_id, index_name=index_name)
print(f"ai_search.definitions: {ai_search.definitions}")
print(f"ai_search.resources: {ai_search.resources}")

ai_search.definitions: [{'type': 'azure_ai_search'}]
ai_search.resources: {'azure_ai_search': {'indexes': [{'index_connection_id': '/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AISearch', 'index_name': 'ms-surface-specs'}]}}


# Create AI Foundry Agent

In [6]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=model_name,
    name="aisearch-agent",
    instructions="You are a helpful assistant",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources,
    headers={"x-ms-enable-preview": "true"},
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_ul8MekPllPkuMrXT8GU4kInQ', 'object': 'assistant', 'created_at': 1738151597, 'name': 'aisearch-agent', 'description': None, 'model': 'gpt-4o-2024-08-06', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'azure_ai_search'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'azure_ai_search': {'indexes': [{'index_connection_id': '/subscriptions/eca2eddb-0f0c-4351-a634-52751499eeea/resourceGroups/mmai04-rg/providers/Microsoft.MachineLearningServices/workspaces/mmai-hub04-prj01-fvye/connections/mmai-hub04-fvye-connection-AISearch', 'index_name': 'ms-surface-specs'}]}}, 'metadata': {}, 'response_format': 'auto'}>

# Create the thread and attach a new message to it

In [7]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id, 
    role="user", 
    content="What kind of keys does the MS Surface Book keyboard include? Do **NOT** use your own internal knowledge.",
)
print(f"Created message: {message}")

Created thread: {'id': 'thread_vruTaHnwYLksA3lQZ89CeXmh', 'object': 'thread', 'created_at': 1738151603, 'metadata': {}, 'tool_resources': {}}

Created message: {'id': 'msg_ynD09J0bCl1wjMkjBiUo8TWt', 'object': 'thread.message', 'created_at': 1738151603, 'assistant_id': None, 'thread_id': 'thread_vruTaHnwYLksA3lQZ89CeXmh', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What kind of keys does the MS Surface Book keyboard include? Do **NOT** use your own internal knowledge.', 'annotations': []}}], 'attachments': [], 'metadata': {}}


# Run the agent syncrhonously

In [8]:
%%time
# Create and process assistant run in thread with tools
run = project_client.agents.create_and_process_run\
    (thread_id=thread.id, assistant_id=agent.id)

print(f"Run finished with status: {run.status}.\n\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED.

Run: {'id': 'run_yt117XVf9mxJ8fpP86d6qjyk', 'object': 'thread.run', 'created_at': 1738151606, 'assistant_id': 'asst_ul8MekPllPkuMrXT8GU4kInQ', 'thread_id': 'thread_vruTaHnwYLksA3lQZ89CeXmh', 'status': 'completed', 'started_at': 1738151606, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1738151612, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-2024-08-06', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'azure_ai_search'}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 1132, 'completion_tokens': 84, 'total_tokens': 1216}, 'response_format': 'auto', 'tool_choice': 'auto', 'parallel_tool_calls': True}
CPU times: total: 15.6 ms
Wall time: 7.92 s


# Fetch messages from the thread after the agent run execution

In [9]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages.data), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

CONTENT 1 (MessageTextContent) --> Text: What kind of keys does the MS Surface Book keyboard include? Do **NOT** use your own internal knowledge.

===== MESSAGE 2 =====

CONTENT 1 (MessageTextContent) --> Text: I couldn't find the specific information in the provided documents. You might want to check the official Microsoft Surface Book specifications or a detailed product review for comprehensive information about the keyboard keys. If you have any other questions or need further assistance, feel free to ask!


In [11]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

Last Message: I couldn't find the specific information in the provided documents. You might want to check the official Microsoft Surface Book specifications or a detailed product review for comprehensive information about the keyboard keys. If you have any other questions or need further assistance, feel free to ask!


# Print annotations from the messages

In [12]:
print(f"Number of annotation(s): {len(last_msg.text.annotations)}")

for annotation in last_msg.text.annotations:
    print(annotation["text"], annotation["file_citation"]["file_id"])

Number of annotation(s): 0


# Run Steps

In [ ]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)

print(f'Nr of run step(s): {len(run_steps["data"])}\n')
i=0
for rs in run_steps["data"]:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

# START Teardown

In [13]:
print(f"deleting trhead: {thread}...")
project_client.agents.delete_thread(thread.id)

deleting trhead: {'id': 'thread_vruTaHnwYLksA3lQZ89CeXmh', 'object': 'thread', 'created_at': 1738151603, 'metadata': {}, 'tool_resources': {}}...


{'id': 'thread_vruTaHnwYLksA3lQZ89CeXmh', 'object': 'thread.deleted', 'deleted': True}

In [14]:
# Delete all agents

print(f"{len(project_client.agents.list_agents()['data'])} agent(s) will now be deleted")

i=0
for pca in project_client.agents.list_agents()['data']:
    i += 1
    project_client.agents.delete_agent(pca.id)
    print(f"\n{i} - Agent {pca.name} has been deleted")

1 agent(s) will now be deleted

1 - Agent aisearch-agent has been deleted


# HIC SUNT LEONES